In [10]:
import numpy as np
import csv
%matplotlib inline
import pandas as pd
from frgpascal.hardware.sampletray import SampleTray, AVAILABLE_VERSIONS as sampletray_versions
from frgpascal.hardware.liquidlabware import LiquidLabware, AVAILABLE_VERSIONS as liquid_labware_versions
from frgpascal.hardware.hotplate import AVAILABLE_VERSIONS as hotplate_versions
from frgpascal.experimentaldesign.helpers import build_sample_list, plot_tray, handle_liquids, samples_to_dataframe
from frgpascal.experimentaldesign.tasks import *
from frgpascal.experimentaldesign.scheduler import Scheduler

from scipy.optimize import nnls

In [15]:
#### name parsing helper functions
def components_to_name(components, delimiter="_"):
    composition_label = ""
    for c, n in components.items():
        if n > 0:
            composition_label += "{0}{1:.2f}{2}".format(c, n, delimiter)

    return composition_label[:-1]


def name_to_components(
    name,
    factor=1,
    delimiter="_",
):
    """
    given a chemical formula, returns dictionary with individual components/amounts
    expected name format = 'MA0.5_FA0.5_Pb1_I2_Br1'.
    would return dictionary with keys ['MA, FA', 'Pb', 'I', 'Br'] and values [0.5,.05,1,2,1]*factor
    """
    components = {}
    for part in name.split(delimiter):
        species = part
        count = 1.0
        for l in range(len(part), 0, -1):
            try:
                count = float(part[-l:])
                species = part[:-l]
                break
            except:
                pass
        if species == "":
            continue
        components[species] = count * factor
    return components


In [59]:
def solutions_to_matrix(solutions:list):
    if isinstance(solutions, Solution):
        solutions = [solutions]

    # get possible solution components from stock list
    components = set()
    for s in solutions:
        components.update(s.solute_dict.keys(), s.solvent_dict.keys())
    components = list(
        components
    )  # sets are not order-preserving, lists are - just safer this way

    # organize components into a stock matrix, keep track of which rows are solvents
    stock_matrix = np.zeros((len(components), len(solutions)))
    solvent_idx = set()
    for n, s in enumerate(solutions):
        for m, c in enumerate(components):
            if c in s.solute_dict:
                stock_matrix[m, n] = s.solute_dict[c] * s.molarity
            elif c in s.solvent_dict:
                stock_matrix[m, n] = s.solvent_dict[c]
                solvent_idx.add(m)
    solvent_idx = list(solvent_idx)

    return stock_matrix, solvent_idx
    

In [31]:
def calculate_mix(
    target: Solution,
    volume: float,
    stock_solutions: list,
    tolerance: float = 0.05,
):
    """
    given a target solution, target volume, and list of stock solutions, calculates
    the volumes needed from individual stocks to achieve target composition

    tolerance (float): maximum error for single site (relative, not absolute) allowed.

    """
    # get possible solution components from stock list
    components = set()
    for s in stock_solutions:
        components.update(s.solute_dict.keys(), s.solvent_dict.keys())
    components = list(
        components
    )  # sets are not order-preserving, lists are - just safer this way

    # organize components into a stock matrix, keep track of which rows are solvents
    stock_matrix = np.zeros((len(components), len(stock_solutions)))
    solvent_idx = set()
    for n, s in enumerate(stock_solutions):
        for m, c in enumerate(components):
            if c in s.solute_dict:
                stock_matrix[m, n] = s.solute_dict[c] * s.molarity
            elif c in s.solvent_dict:
                stock_matrix[m, n] = s.solvent_dict[c]
                solvent_idx.add(m)
    solvent_idx = list(solvent_idx)

    # organize target solution into a matrix of total mols desired of each component
    target_matrix = np.zeros((len(components),))
    for m, c in enumerate(components):
        if c in target.solute_dict:
            target_matrix[m] = target.solute_dict[c] * target.molarity * volume
        elif c in target.solvent_dict:
            target_matrix[m] = target.solvent_dict[c] * volume

    # solve for the mixture amounts
    amount_matrix, *data = nnls(
        stock_matrix, target_matrix, maxiter=1e3
    )  # volumes to mix. math is better if not such small values in matrix, so scale to uL for calculation
    amount_matrix[
        amount_matrix < 1
    ] = 0  # clean up values that are essentially 0. If we have a significant negative value here, should get caught downstream
    amount_matrix = np.round(
        amount_matrix,1
    )  # round to nearest uL (values are in L at this point)

    # double check that the solved amounts make sense
    doublecheck = stock_matrix @ amount_matrix
    doublecheck[solvent_idx] /= (
        doublecheck[solvent_idx].sum() / volume
    )  # solvents should sum to one
    # print(stock_matrix)
    # print(amount_matrix)
    # print(target_matrix)
    composition_error = max(
        [np.abs(1 - c / t) for c, t in zip(doublecheck, target_matrix) if t > 0]
    )  # max single-component error fraction

    if (
        composition_error < tolerance
    ):  # check that we are within error tolerance wrt target composition AT EACH SOLUTE/SOLVENT SPECIES
        return amount_matrix
    else:
        solute = components_to_name(
            {
                c: amt / target.molarity / volume
                for c, amt in zip(components, doublecheck)
                if amt > 0 and c not in target.solvent_dict
            }
        )
        solvent = components_to_name(
            {
                c: amt / target.molarity / volume
                for c, amt in zip(components, doublecheck)
                if amt > 0 and c in target.solvent_dict
            }
        )
        # print(solvent)
        raise Exception(
            f"Unable to generate target solution ({volume} uL of {target}) with current stock solutions.\n\n"
            f"Closest match ({volume} uL of {target.molarity}M {solute} in {solvent}) has a max site error of {composition_error*100:.2f}%"
        )  # {Off by {composition_error*100:.2f}%%')



In [32]:
target_solutions = [
    Solution(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA0.25_MA0.75_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA0.75_MA0.25_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='MA0.5_FA0.5_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
]


In [33]:
stock_solutions = [
    Solution(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solvent='DMF',
    ),
    Solution(
        solvent='DMSO',
    ),
    Solution(
        solvent='Chlorobenzene',
    ),
    Solution(
        solvent='MethylAcetate',
    ),
    Solution(
        solutes='spiro',
        solvent='IPA',
        molarity=1
    ),
]

In [50]:
target = Solution(
        solutes='MA0.05_FA0.95_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    )

In [51]:
target_solutions

[<Solution>1M MA_Pb_I3 in DMF9_DMSO1,
 <Solution>1M FA_Pb_I3 in DMF9_DMSO1,
 <Solution>1M FA0.25_MA0.75_Pb_I3 in DMF9_DMSO1,
 <Solution>1M FA0.75_MA0.25_Pb_I3 in DMF9_DMSO1,
 <Solution>1M MA0.5_FA0.5_Pb_I3 in DMF9_DMSO1]

In [55]:
target

<Solution>1M MA0.05_FA0.95_Pb_I3 in DMF9_DMSO1

In [58]:
calculate_mix(
    target=target,
    stock_solutions=[target_solutions[i] for i in [1,3]],
    volume=100)

array([80., 20.])

In [60]:
solutions_to_matrix(
    solutions=target_solutions,
)

IndexError: index 5 is out of bounds for axis 1 with size 5